In [1]:
using DataFrames, TableOperations, Tables, Random
using Recommenders: Movielens100k, load_dataset, ratio_split, BPR, evaluate_u2i, MeanPrecision, MeanRecall, MeanNDCG, fit!, predict_u2i, make_u2i_dataset

┌ Info: Precompiling Recommenders [f1dcbcfb-0b1d-45c0-883e-2edcfc1d4c23]
└ @ Base loading.jl:1317


In [2]:
ml100k = Movielens100k()
download(ml100k)
rating, user, movie = load_dataset(ml100k);

In [3]:
Random.seed!(1234);
train_valid_table, test_table = ratio_split(rating, 0.8)

train_table, valid_table = ratio_split(train_valid_table, 0.8)
length(Tables.rows(train_table)), length(Tables.rows(valid_table)), length(Tables.rows(test_table))

(64000, 16000, 20000)

In [4]:
prec10 = MeanPrecision(10)
recall10 = MeanRecall(10)
ndcg10 = MeanNDCG(10)
metrics = [prec10, recall10, ndcg10]

3-element Vector{Recommenders.MeanMetric}:
 Recommenders.MeanMetric{Recommenders.Precision}(Recommenders.Precision(10, "precision"))
 Recommenders.MeanMetric{Recommenders.Recall}(Recommenders.Recall(10, "recall"))
 Recommenders.MeanMetric{Recommenders.NDCG}(Recommenders.NDCG(10, "ndcg"))

In [5]:
model = BPR(2^8, 0.021362893548487288)

BPR(256, Recommenders.BPRLoss(), 0.021362893548487288, nothing, nothing, nothing, nothing, nothing, nothing)

In [11]:
evaluate_u2i(model, train_table, valid_table, metrics, 10, col_item=:movieid, tolerance=1e-12, learning_rate=0.020212854924314114, drop_history=true, verbose=1000)

pos_itner_idx = 1:64000


┌ Info: n_sample=1000: train_loss=0.3057343543204828, delta=-0.0002191825911797229, cond=true
└ @ Recommenders /Users/keisuke.yanagi/workspace/Recommender.jl/src/model/bpr.jl:112
┌ Info: n_sample=2000: train_loss=0.26520857906346174, delta=-0.00010813899239590435, cond=true
└ @ Recommenders /Users/keisuke.yanagi/workspace/Recommender.jl/src/model/bpr.jl:112
┌ Info: n_sample=3000: train_loss=0.2417938239947497, delta=5.451312885568682e-5, cond=true
└ @ Recommenders /Users/keisuke.yanagi/workspace/Recommender.jl/src/model/bpr.jl:112
┌ Info: n_sample=4000: train_loss=0.2280358437792299, delta=-4.384245838373439e-5, cond=true
└ @ Recommenders /Users/keisuke.yanagi/workspace/Recommender.jl/src/model/bpr.jl:112
┌ Info: n_sample=5000: train_loss=0.21560495386104772, delta=-4.2289946057705016e-5, cond=true
└ @ Recommenders /Users/keisuke.yanagi/workspace/Recommender.jl/src/model/bpr.jl:112
┌ Info: n_sample=6000: train_loss=0.20609544510883224, delta=-1.6769096088045457e-5, cond=true
└ @ Recomm

(ndcg10 = 0.08283624719682874, precision10 = 0.07680851063829801, recall10 = 0.04826671623651372)

In [8]:
train_valid_table = DataFrame(train_valid_table)
train_valid_table = train_valid_table[shuffle(1:size(train_valid_table, 1)),:];

In [9]:
model = BPR(2^8, 0.021362893548487288)

BPR(256, Recommenders.BPRLoss(), 0.021362893548487288, nothing, nothing, nothing, nothing, nothing, nothing)

In [10]:
evaluate_u2i(model, train_valid_table, test_table, metrics, 10, col_item=:movieid, n_epochs=256, learning_rate=0.020212854924314114, drop_history=true, verbose=2)

┌ Info: epoch=2: train_loss=0.567572800500325, val_metric=0.0, best_val_metric=0.0, best_epoch=1
└ @ Recommenders /Users/keisuke.yanagi/workspace/Recommender.jl/src/model/bpr.jl:133
┌ Info: epoch=4: train_loss=0.42234122608840663, val_metric=0.0, best_val_metric=0.0, best_epoch=1
└ @ Recommenders /Users/keisuke.yanagi/workspace/Recommender.jl/src/model/bpr.jl:133
┌ Info: epoch=6: train_loss=0.3473923702235575, val_metric=0.0, best_val_metric=0.0, best_epoch=1
└ @ Recommenders /Users/keisuke.yanagi/workspace/Recommender.jl/src/model/bpr.jl:133
┌ Info: epoch=8: train_loss=0.30204838601298706, val_metric=0.0, best_val_metric=0.0, best_epoch=1
└ @ Recommenders /Users/keisuke.yanagi/workspace/Recommender.jl/src/model/bpr.jl:133
┌ Info: epoch=10: train_loss=0.27496139220217874, val_metric=0.0, best_val_metric=0.0, best_epoch=1
└ @ Recommenders /Users/keisuke.yanagi/workspace/Recommender.jl/src/model/bpr.jl:133
┌ Info: epoch=12: train_loss=0.2543164647066053, val_metric=0.0, best_val_metric=0

(ndcg10 = 0.40596797549310076, precision10 = 0.34750795334040285, recall10 = 0.2240853245895258)

In [15]:
Tables.getcolumn(test_table, :userid)

20000-element Vector{Int64}:
 936
 894
  46
  51
 161
  43
  69
 835
 393
 606
 771
 407
 325
   ⋮
 328
 881
 405
 230
 472
 495
 246
 262
 537
 405
  62
 465